In [2]:
# 전처리 코드 정의
def clean_review_column(df, column_name):
    # '\n'을 제거하고 한글 자모 및 구두점 제거
    df[column_name] = df[column_name].str.replace('\n', ' ', regex=False)  # 개행 문자 제거
    df[column_name] = df[column_name].str.replace(r'[ㄱ-ㅎㅏ-ㅣ]+', ' ', regex=True)  # 한글 자모 제거
    df[column_name] = df[column_name].str.replace(r'[^\w\s]', ' ', regex=True)  # 특정 구두점 제거
    
    # 여러 공백을 한 칸으로 변환
    df[column_name] = df[column_name].str.replace(r'\s+', ' ', regex=True)  # 여러 개의 공백을 한 칸으로 변환

    return df

In [3]:
import pandas as pd

df = pd.read_csv('[SET] 워셔블 케이블 반팔 니트 세트.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

In [4]:
documents

0      여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...
1                   여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요
2      시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...
3      무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...
4         최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요
                             ...                        
885                          합리적인 가격이고 여름 반팔은 역시나 수아레네요 
886                         옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요
887           약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요
888                      사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다
889                       가성비 젛아요 품은 생각보다 좀 커요 참고하세요 이뻐요
Name: Review, Length: 890, dtype: object

# 12

In [87]:
df = pd.read_csv('reviews_2551401.csv')
df = clean_review_column(df, 'Review')
documents = df['Review']

from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

                                              Review  \
0  168 기준 살짝 짧은 기장이기는 하지만 속바지가 있어서 편하게 입기 좋아요 옆쪽 ...   
1  디자인은 정말 예쁜데 안에 박음질은 엉망이에요 단 기장이 아예 다르게 어긋나있는데도...   
2              교복 치마 대신 입으려고 샀는데 편하고 핏도 예쁘고 너무 만족합니다   
3  저렴한 가격인데 품질 좋네요 안에 속바지가 있어서 좀 짧긴해도 부담없어요 저한테는 ...   
4  사이즈랑 길이가 완전 딱입니다 160대초반 44사이즈 입으시면 xs사이즈 예쁘게 잘...   

                                            Keywords  
0  [(기장이기는, 0.6006), (심심하지, 0.5184), (속바지가, 0.490...  
1  [(엉망이에요, 0.697), (허접할거, 0.6658), (기장이, 0.6189)...  
2  [(만족합니다, 0.7323), (입으려고, 0.6412), (편하고, 0.6041...  
3  [(엉덩이가, 0.667), (조여서, 0.6089), (저렴한, 0.5945), ...  
4  [(xs사이즈, 0.5646), (44사이즈, 0.5194), (입으시면, 0.42...  


In [88]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입', '생각', '하다', '기다', '들다', '안']

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('편하다', 253), ('바지', 186), ('사이즈', 175), ('허리', 118), ('길이', 61), ('예쁘다', 56), ('만족하다', 53), ('엉덩이', 53), ('적당하다', 52), ('기장', 49)]


In [94]:
csv_title = "reviews_2551401"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
new_row = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

df = pd.read_csv('output.csv')
df = pd.concat([df, new_row], ignore_index=True)
df.to_csv('output.csv', index=False)

In [95]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


In [99]:
df = df.drop(index=12)

In [101]:
df = df.reset_index(drop=True)
df.to_csv('output.csv', index=False)

In [102]:
df

,title,keyword
0,[SET] 워셔블 케이블 반팔 니트 세트,"사이즈, 색감, 여름, 가성 비, 재질, 가격, 배송, 니트, 만족하다, 시원하다"
1,DOODLE HEART HALF T WHITE GREYISH BLUE,"프린팅, 디자인, 사이즈, 이쁘다, 들다, 편하다, 배송, 예쁘다, 재질, 생각"
2,[2PACK] 쿨 코튼 티셔츠 블랙+화이트,"사이즈, 편하다, 시원하다, 재질, 여름, 배송, 티, 적당하다, 가격, 가성 비"
3,아우터3. ASI 포시즌 에센셜 피그먼트 후드집업_차콜 그레이,"색감, 사이즈, 배송, 생각, 지퍼, 편하다, 디자인, 이쁘다, 적당하다, 무겁다"
4,아우터2. 파이어버드 트랙탑 - 블랙 IJ7058,"사이즈, 편하다, 배송, 아디다스, 적당하다, 이쁘다, 만족하다, 크다, 져지, 디자인"
5,42283 루즈핏 워크쇼츠 Black_reviews.csv,"사이즈, 허리, 디 키즈, 재질, 바지, 크다, 만족하다, 업 하다, 기장, 핏"
6,(여름버전추가) 투턱 와이드 롱 스웨트 팬츠 [그레이]_reviews,"편하다, 여름, 기장, 바지, 배송, 사이즈, 핏, 시원하다, 이쁘다, 만족하다"
7,아우터1. 베켄바우어 트랙탑 - 인디고 IP0418,"사이즈, 아디다스, 편하다, 색감, 예쁘다, 만족하다, 이쁘다, 실물, 역시, 기장"
8,1967 JET BLACK JEANS [WIDE STRAIGHT]_reviews,"기장, 바지, 사이즈, 색감, 만족하다, 적당하다, 브랜디 드, 핏, 이쁘다, 허리"
9,reviews_1875663,"시원하다, 길이, 사이즈, 편하다, 적당하다, 비치다, 원피스, 배송, 재질, 라인"


# Key BERT

In [5]:
from keybert import KeyBERT
from sklearn.feature_extraction.text import CountVectorizer

# KeyBERT 모델 초기화
kw_model = KeyBERT()

vectorizer = CountVectorizer(ngram_range=(1, 1))

# 각 문서에서 핵심 키워드 추출
def extract_keywords(text):
    # 각 텍스트에서 상위 5개의 키워드 추출
    keywords = kw_model.extract_keywords(text, vectorizer=vectorizer, top_n=5)
    return keywords

# 전체 문서에 대해 핵심 키워드 추출
df['Keywords'] = documents.apply(extract_keywords)

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())

C:\Users\HWAN\anaconda3\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange



                                              Review  \
0  여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...   
1               여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요   
2  시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...   
3  무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...   
4     최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요   

                                            Keywords  
0  [(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0....  
1  [(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)...  
2  [(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ...  
3  [(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594...  
4  [(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),...  


In [6]:
df

,Review,Star,Keywords
0,여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...,5.0,"[(정사이즈로, 0.3535), (구매하시면, 0.3386), (구매하시거나, 0...."
1,여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요,5.0,"[(입어도, 0.6102), (부드러워서, 0.4709), (여름에, 0.4678)..."
2,시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...,5.0,"[(적당히, 0.6256), (라인이, 0.6139), (시원한, 0.6104), ..."
3,무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...,5.0,"[(재질확인후이, 0.7081), (방문해서, 0.6456), (무신사, 0.594..."
4,최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요,5.0,"[(스펙이랑, 0.6513), (사이즈, 0.5276), (사세요, 0.4755),..."
...,...,...,...
885,합리적인 가격이고 여름 반팔은 역시나 수아레네요,5.0,"[(합리적인, 0.6039), (가격이고, 0.5589), (반팔은, 0.4801)..."
886,옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요,5.0,"[(선택하시길, 0.6377), (2벌, 0.521), (다른거로, 0.5048),..."
887,약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요,5.0,"[(한여름에는, 0.6743), (무난, 0.6062), (합니다, 0.5799),..."
888,사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다,5.0,"[(추천합니다, 0.8154), (다니고, 0.6586), (사이즈도, 0.6375..."


In [7]:
from collections import Counter

# 각 문서에서 추출된 키워드를 모은 리스트 생성
all_keywords = []

for keywords in df['Keywords']:
    # keywords는 [(단어, 가중치), (단어, 가중치), ...] 형식이므로 단어만 추출
    all_keywords.extend([keyword[0] for keyword in keywords])

# 키워드의 빈도 계산
keyword_counts = Counter(all_keywords)

# 빈도 상위 10개의 키워드 출력
top_keywords = keyword_counts.most_common(15)

# 상위 10개의 키워드와 그 빈도를 출력
print("상위 15개의 키워드:")
for word, count in top_keywords:
    print(f"{word}: {count}번")

상위 15개의 키워드:
너무: 91번
입기: 60번
가성비: 49번
좋아요: 46번
여름에: 44번
사이즈도: 44번
만족합니다: 41번
색감도: 39번
입고: 35번
생각보다: 34번
사이즈: 30번
마음에: 28번
입을: 27번
재질도: 27번
좋고: 27번


In [8]:
from collections import Counter
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 원형으로 변환
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']] # 명사, 동사, 형용사 추출
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 키워드를 모두 모아서 리스트로 풀기
all_keywords = []
for keywords in df['Keywords']:
    # 키워드 원형으로 변환 후 리스트에 추가 (불용어 제외)
    for keyword, score in keywords:
        lemmatized_keyword = lemmatize_keyword(keyword)
        if lemmatized_keyword and lemmatized_keyword not in stop_words:  # 불용어 필터링
            all_keywords.append(lemmatized_keyword)

# 각 키워드의 빈도를 계산
keyword_counts = Counter(all_keywords)

# 상위 10개의 키워드 추출
top_10_keywords = keyword_counts.most_common(10)

# 결과 출력
print(top_10_keywords)

[('사이즈', 114), ('색감', 92), ('여름', 72), ('가성 비', 62), ('재질', 59), ('가격', 58), ('배송', 56), ('니트', 55), ('만족하다', 53), ('시원하다', 52)]


In [23]:
csv_title = "[SET] 워셔블 케이블 반팔 니트 세트"

# 키워드만 추출하여 리스트로 변환
top_10_keywords_list = [keyword for keyword, _ in top_10_keywords]

keywords_str = ', '.join(top_10_keywords_list)

# 데이터프레임 생성 (2열 1행: 'title'과 'keyword')
df = pd.DataFrame({
    'title': [csv_title],
    'keyword': [keywords_str]
})

In [25]:
df.to_csv('output.csv', index=False)

# KLUE

In [4]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# KLUE BERT 모델과 토크나이저 로드
model = BertModel.from_pretrained('klue/bert-base')
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

def get_embedding(text):
    # 텍스트 토크나이징
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # BERT 모델로 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)
    
    # CLS 토큰의 임베딩 추출
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    
    return cls_embedding.numpy()

def extract_keywords_klueBERT(text, top_n=5):
    # 문장을 토큰화
    words = text.split()
    
    # 각 단어의 임베딩 추출
    word_embeddings = [get_embedding(word) for word in words]
    
    # 문서 전체의 임베딩 추출
    document_embedding = get_embedding(text)
    
    # 코사인 유사도를 통해 문서와 단어 간 유사도를 계산
    similarities = [cosine_similarity([document_embedding], [we])[0][0] for we in word_embeddings]
    
    # 유사도 기준으로 상위 n개의 키워드를 선택
    keywords = sorted(zip(words, similarities), key=lambda x: x[1], reverse=True)[:top_n]
    
    return [word for word, _ in keywords]

# DataFrame에 적용
df['Keywords'] = df['Review'].apply(lambda x: extract_keywords_klueBERT(x, top_n=5))

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())


C:\Users\HWAN\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


                                              Review  \
0  여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...   
1               여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요   
2  시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...   
3  무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...   
4     최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요   

                            Keywords  
0       [사이즈는, 괜찮아요, 재질도, 퀄리티랑, 배송은]  
1       [두께에요, 부드러워서, 괜찮아요, 입어도, 입기]  
2         [편하네요, 좋습니다, 좋아요, 예쁘게, 단추]  
3  [두께감입니다, 구매하였습니다, 핏도, 맘에, 재질확인후이]  
4     [사세요, 최고에요, 좋야요, 비슷하시면, 오버핏인데]  


In [7]:
from konlpy.tag import Okt

okt = Okt()

def okt_morphs(text):
    # morphs를 사용하고 stem=True로 어간 추출
    return okt.morphs(text, stem=True)

# 키워드 추출 결과를 Okt 형태소로 정리
df['Okt_Morphs'] = df['Keywords'].apply(lambda x: [okt_morphs(kw) for kw in x])
df

,Review,Star,Keywords,Okt_Morphs
0,여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...,5.0,"[사이즈는, 괜찮아요, 재질도, 퀄리티랑, 배송은]","[[사이즈, 는], [괜찮다], [재질, 도], [퀄리티, 랑], [배송, 은]]"
1,여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요,5.0,"[두께에요, 부드러워서, 괜찮아요, 입어도, 입기]","[[두께, 에요], [부드럽다], [괜찮다], [입다], [입다]]"
2,시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...,5.0,"[편하네요, 좋습니다, 좋아요, 예쁘게, 단추]","[[편하다], [좋다], [좋다], [예쁘다], [단추]]"
3,무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...,5.0,"[두께감입니다, 구매하였습니다, 핏도, 맘에, 재질확인후이]","[[두께, 감, 이다], [구매, 하다], [핏, 도], [맘, 에], [재질, 확..."
4,최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요,5.0,"[사세요, 최고에요, 좋야요, 비슷하시면, 오버핏인데]","[[사다], [최고, 에요], [좋다], [비슷하다], [오, 버핏, 인데]]"
...,...,...,...,...
885,합리적인 가격이고 여름 반팔은 역시나 수아레네요,5.0,"[수아레네요, 역시나, 반팔은, 가격이고, 여름]","[[수아, 레, 네, 요], [역시, 나], [반, 팔, 은], [가격, 이고], ..."
886,옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요,5.0,"[선택하실때, 선택하시길, 바래요, 다른거로, 2벌]","[[선택, 하다, 때], [선택, 하다], [바, 래, 요], [다른, 거, 로],..."
887,약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요,5.0,"[힘들듯해요, 이정도면, 무난, 느낌이긴, 합니다]","[[힘들다, 해, 요], [이, 정도, 면], [무난], [느끼다], [하다]]"
888,사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다,5.0,"[추천합니다, 사이즈도, 있습니다, 좋고, 입고]","[[추천, 하다], [사이즈, 도], [있다], [좋다], [입다]]"


In [8]:
from konlpy.tag import Okt

# Okt 형태소 분석기 초기화
okt = Okt()

# 불용어 리스트
stop_words = ['좋다', '입다', '같다', '이다', '있다', '입']

# 키워드를 원형으로 변환
def lemmatize_keyword(keyword):
    morphs = okt.pos(keyword, stem=True)  # 형태소 분석 및 어간 추출
    lemmatized = [word for word, pos in morphs if pos in ['Noun', 'Verb', 'Adjective']] # 명사, 동사, 형용사 추출
    lemmatized = [word for word in lemmatized if word not in stop_words]  # 불용어 제거
    return ' '.join(lemmatized) if lemmatized else None  # 원형 변환된 단어들을 합침, 없으면 None 반환

# 데이터프레임의 키워드에 대해 lemmatize 함수 적용
df['Lemmatized_Keywords'] = df['Keywords'].apply(lambda x: [lemmatize_keyword(kw) for kw in x if lemmatize_keyword(kw)])

# 결과 확인
print(df[['Review', 'Keywords', 'Lemmatized_Keywords']].head())


                                              Review  \
0  여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...   
1               여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요   
2  시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...   
3  무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...   
4     최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요   

                            Keywords           Lemmatized_Keywords  
0       [사이즈는, 괜찮아요, 재질도, 퀄리티랑, 배송은]       [사이즈, 괜찮다, 재질, 퀄리티, 배송]  
1       [두께에요, 부드러워서, 괜찮아요, 입어도, 입기]               [두께, 부드럽다, 괜찮다]  
2         [편하네요, 좋습니다, 좋아요, 예쁘게, 단추]                [편하다, 예쁘다, 단추]  
3  [두께감입니다, 구매하였습니다, 핏도, 맘에, 재질확인후이]  [두께 감, 구매 하다, 핏, 맘, 재질 확인 후]  
4     [사세요, 최고에요, 좋야요, 비슷하시면, 오버핏인데]            [사다, 최고, 비슷하다, 버핏]  


In [9]:
df

,Review,Star,Keywords,Okt_Morphs,Lemmatized_Keywords
0,여름에 꼭 가지고 있어야 템 중 하나 입니다 원플원임에도 불구하고 퀄리티랑 핏이 너...,5.0,"[사이즈는, 괜찮아요, 재질도, 퀄리티랑, 배송은]","[[사이즈, 는], [괜찮다], [재질, 도], [퀄리티, 랑], [배송, 은]]","[사이즈, 괜찮다, 재질, 퀄리티, 배송]"
1,여름에 입기 딱 좋은 두께에요 부드러워서 맨 몸에 입어도 괜찮아요,5.0,"[두께에요, 부드러워서, 괜찮아요, 입어도, 입기]","[[두께, 에요], [부드럽다], [괜찮다], [입다], [입다]]","[두께, 부드럽다, 괜찮다]"
2,시원한 느낌이라 좋습니다 부드러워 편하네요 카라와 단추 라인이 예쁘게 잘 나왔습니다...,5.0,"[편하네요, 좋습니다, 좋아요, 예쁘게, 단추]","[[편하다], [좋다], [좋다], [예쁘다], [단추]]","[편하다, 예쁘다, 단추]"
3,무신사 매장까지 방문해서 재질확인후이 구매하였습니다 여름에 입기 좋은 두께감입니다 ...,5.0,"[두께감입니다, 구매하였습니다, 핏도, 맘에, 재질확인후이]","[[두께, 감, 이다], [구매, 하다], [핏, 도], [맘, 에], [재질, 확...","[두께 감, 구매 하다, 핏, 맘, 재질 확인 후]"
4,최고에요 사이즈 살짝 오버핏인데 그냥 너무 좋야요 제 스펙이랑 비슷하시면 m 사세요,5.0,"[사세요, 최고에요, 좋야요, 비슷하시면, 오버핏인데]","[[사다], [최고, 에요], [좋다], [비슷하다], [오, 버핏, 인데]]","[사다, 최고, 비슷하다, 버핏]"
...,...,...,...,...,...
885,합리적인 가격이고 여름 반팔은 역시나 수아레네요,5.0,"[수아레네요, 역시나, 반팔은, 가격이고, 여름]","[[수아, 레, 네, 요], [역시, 나], [반, 팔, 은], [가격, 이고], ...","[수아 레, 역시, 팔, 가격, 여름]"
886,옷 2벌 와요 선택하실때 다른거로 선택하시길 바래요,5.0,"[선택하실때, 선택하시길, 바래요, 다른거로, 2벌]","[[선택, 하다, 때], [선택, 하다], [바, 래, 요], [다른, 거, 로],...","[선택 하다 때, 선택 하다, 바 요, 거, 벌]"
887,약간 큰 느낌이긴 한데 이 가격에 이정도면 무난 합니다 한여름에는 힘들듯해요,5.0,"[힘들듯해요, 이정도면, 무난, 느낌이긴, 합니다]","[[힘들다, 해, 요], [이, 정도, 면], [무난], [느끼다], [하다]]","[힘들다 해, 정도, 무난, 느끼다, 하다]"
888,사이즈도 좋고 여름에 잘 입고 다니고 있습니다 추천합니다,5.0,"[추천합니다, 사이즈도, 있습니다, 좋고, 입고]","[[추천, 하다], [사이즈, 도], [있다], [좋다], [입다]]","[추천 하다, 사이즈]"


# KoBERT

In [ ]:
from transformers import BertModel, BertTokenizer
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# koBERT 모델과 토크나이저 로드
model = BertModel.from_pretrained('skt/kobert-base-v1')
tokenizer = BertTokenizer.from_pretrained('skt/kobert-base-v1')

def get_embedding(text):
    # 텍스트 토크나이징
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True)
    
    # koBERT 모델로 임베딩 추출
    with torch.no_grad():
        outputs = model(**inputs)
    
    # CLS 토큰의 임베딩 추출
    cls_embedding = outputs.last_hidden_state[:, 0, :].squeeze()
    
    return cls_embedding.numpy()

def extract_keywords_koBERT(text, top_n=5):
    # 문장을 토큰화
    words = text.split()
    
    # 각 단어의 임베딩 추출
    word_embeddings = [get_embedding(word) for word in words]
    
    # 문서 전체의 임베딩 추출
    document_embedding = get_embedding(text)
    
    # 코사인 유사도를 통해 문서와 단어 간 유사도를 계산
    similarities = [cosine_similarity([document_embedding], [we])[0][0] for we in word_embeddings]
    
    # 유사도 기준으로 상위 n개의 키워드를 선택
    keywords = sorted(zip(words, similarities), key=lambda x: x[1], reverse=True)[:top_n]
    
    return [word for word, _ in keywords]

# DataFrame에 적용
df['Keywords'] = df['Review'].apply(lambda x: extract_keywords_koBERT(x, top_n=5))

# 키워드가 잘 추출되었는지 확인
print(df[['Review', 'Keywords']].head())
